<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [102]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [105]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [107]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

675
173


In [108]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#process_tournament(1526)


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [109]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)






  0%|          | 0/675 [00:00<?, ?it/s]




  0%|          | 1/675 [00:00<01:20,  8.34it/s]




  0%|          | 2/675 [00:02<09:13,  1.22it/s]




  0%|          | 3/675 [00:04<13:27,  1.20s/it]




  1%|          | 4/675 [00:06<17:02,  1.52s/it]




  1%|          | 5/675 [00:08<18:24,  1.65s/it]




  1%|          | 7/675 [00:09<13:32,  1.22s/it]




  1%|          | 8/675 [00:09<09:58,  1.11it/s]




  1%|▏         | 9/675 [00:09<07:56,  1.40it/s]




  1%|▏         | 10/675 [00:10<06:24,  1.73it/s]




  2%|▏         | 11/675 [00:10<05:13,  2.12it/s]




  2%|▏         | 12/675 [00:10<04:03,  2.72it/s]




  2%|▏         | 13/675 [00:11<05:57,  1.85it/s]




  2%|▏         | 14/675 [00:14<15:49,  1.44s/it]




  2%|▏         | 16/675 [00:14<11:19,  1.03s/it]




  3%|▎         | 17/675 [00:15<08:14,  1.33it/s]




  3%|▎         | 19/675 [00:15<05:58,  1.83it/s]




  3%|▎         | 22/675 [00:15<04:18,  2.53it/s]




  4%|▎         | 24/675 [00:16<04:54,  2.21it/s]




  4%|

<2 categories , skipping







100%|█████████▉| 673/675 [02:00<00:00,  4.18it/s]




100%|█████████▉| 673/675 [02:13<00:00,  4.18it/s]




100%|██████████| 675/675 [02:44<00:00,  4.11it/s]


In [110]:
ranking = [(k, v[0]/v[1],v[1]) for k,v in all_results.items() if v[1] != 0]
print(ranking)

ranking.sort(key = lambda x: -x[1])

print(ranking)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [111]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)
  
players_ranking = [ x for x in ranking if is_player(x[0]) ]
blank_rank = stat.mean ([ x[1] for x in ranking if is_player(x[0])])
print(blank_rank)

def get_team_ranking(players):
  result = []
  print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      print(player, ' rating is ', all_results[player][0])
      result.append(all_results[player][0])
    else:
      result.append(blank_rank)
  return stat.mean(result)


-0.5379797641338633


In [113]:
spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(i)
    i+=1
  print ('Spearman: ',-stats.spearmanr(rating, ranking)[0])
  print ('Pearson: ',-stats.pearsonr(rating,ranking)[0])
  # ставим минус т.к. у нас места возрастают а сила убывает
  spearman.append(-stats.spearmanr(rating, ranking)[0])
  pearson.append(-stats.pearsonr(rating,ranking)[0])



Streaming output truncated to the last 5000 lines.
69936  rating is  39.4642382967512
88616  rating is  34.58636165661249
91358  rating is  39.139064782317604
115555  rating is  50.26099295224087
Team: [10695, 19915, 23178, 26911, 74382]
10695  rating is  59.4353370728534
19915  rating is  105.30752772933126
23178  rating is  82.40669645061763
26911  rating is  50.67157731588509
74382  rating is  103.22978902315457
Team: [39185, 90800, 101725, 104901, 104902, 113694]
39185  rating is  16.719471264242
90800  rating is  30.527475938236563
101725  rating is  16.18026854096183
104901  rating is  17.38648130146665
104902  rating is  10.376740596863327
113694  rating is  20.562393315251363
Team: [2115, 67807, 111656, 116051, 191655]
2115  rating is  36.44031337826896
67807  rating is  48.88126696035403
111656  rating is  48.269738457728835
116051  rating is  0.08860239857012187
191655  rating is  -2.489916102824292
Team: [19158, 39366, 51852, 132228, 134429, 137380, 176349, 196178]
19158  ra

In [114]:
print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))

[0.6145215233574279, 0.7385917350666544, 0.7449103277674706, 0.5996802805291159, 0.2908784487161796, 0.5755041906908022, 0.36014063562191684, 0.5762198654810187, 0.44450929116538473, 0.4211511694161951, 0.424774016298156, 0.8150183525925432, 0.584316001951095, 0.5272545611873939, 0.6426803890194626, 0.4954907363592675, 0.6593137254901962, 0.7237166180132254, 0.7198945215483132, 0.7266746698679472, 0.7860064310412096, 0.7295443539511336, 0.7067299337358046, 0.7602520252025202, 0.5712992258031315, 0.5935347384767484, 0.4754164531788881, 0.45739130434782604, 0.738324326503369, 0.7828387096774192, 0.8595370147094284, 0.8232552563353265, 0.3790932571685817, 0.7597153804050356, 0.7203007518796991, 0.7584829574924954, 0.3571428571428571, 0.6479454457440096, 0.6011755755701883, 0.6585600510600511, 0.818815331010453, 0.6840815701132453, 0.7272727272727274, 0.9009304871373838, 0.6916779832832851, 0.6274891774891775, 0.7952737823917678, 0.7763397856342985, 0.7518038565208375, 0.8295018658174077, 